<a href="https://colab.research.google.com/github/HaqTetsuya/random/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ComfyUI Complete Setup - Anime Generation with LoRA, Refinement & Embeddings

# ============================================
# STEP 1: Install Dependencies
# ============================================
!pip install -q xformers==0.0.20 triton==2.0.0
!apt-get -y install -qq aria2

# Clone ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI
%cd ComfyUI
!pip install -q -r requirements.txt

# ============================================
# STEP 2: Download Anime Checkpoint Models
# ============================================
print("📦 Downloading Checkpoint Models...")

# Main anime model - Anything V5
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/ckpt/anything-v4.5-vae-swapped/resolve/main/anything-v4.5-vae-swapped.safetensors \
  -d /content/ComfyUI/models/checkpoints -o anything-v5.safetensors

# Alternative: Counterfeit V3 (another popular anime model)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp16.safetensors \
  -d /content/ComfyUI/models/checkpoints -o counterfeit-v3.safetensors

# ============================================
# STEP 3: Download VAE for Better Colors
# ============================================
print("🎨 Downloading VAE...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors \
  -d /content/ComfyUI/models/vae -o vae-ft-mse.safetensors

# ============================================
# STEP 4: Download LoRA Models
# ============================================
print("✨ Downloading LoRA Models...")

# Detail Tweaker LoRA
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://civitai.com/api/download/models/62833 \
  -d /content/ComfyUI/models/loras -o detail-tweaker.safetensors

# Add More Details LoRA
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://civitai.com/api/download/models/87153 \
  -d /content/ComfyUI/models/loras -o add-detail.safetensors

# Anime Lineart Style
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://civitai.com/api/download/models/16134 \
  -d /content/ComfyUI/models/loras -o anime-lineart.safetensors

# ============================================
# STEP 5: Download Negative Embeddings
# ============================================
print("🚫 Downloading Negative Embeddings...")

# EasyNegative
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/embed/EasyNegative/resolve/main/EasyNegative.safetensors \
  -d /content/ComfyUI/models/embeddings -o EasyNegative.safetensors

# Bad Hands 5
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt \
  -d /content/ComfyUI/models/embeddings -o bad-hands-5.pt

# Bad Artist Anime
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/nick-x-hacker/bad-artist/resolve/main/bad-artist-anime.pt \
  -d /content/ComfyUI/models/embeddings -o bad-artist-anime.pt

# ============================================
# STEP 6: Download Upscaler Models for Refinement
# ============================================
print("🔍 Downloading Upscaler Models...")

# 4x-AnimeSharp Upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/Kim2091/AnimeSharp/resolve/main/4x-AnimeSharp.pth \
  -d /content/ComfyUI/models/upscale_models -o 4x-AnimeSharp.pth

# ESRGAN 4x Anime
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_foolhardy_Remacri.pth \
  -d /content/ComfyUI/models/upscale_models -o 4x-foolhardy-Remacri.pth

# ============================================
# STEP 7: Install Cloudflared & Start Server
# ============================================
import subprocess
import threading
import time
import socket

def iframe_thread(port):
  while True:
    time.sleep(0.5)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', port))
    if result == 0:
      break
    sock.close()
  print("\n🚀 ComfyUI loaded! Starting cloudflared tunnel...")
  p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("=" * 60)
      print("🌐 ComfyUI URL:", l[l.find("http"):], end='')
      print("=" * 60)

!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
!python main.py --dont-print-server --listen 0.0.0.0

# ============================================
# ADVANCED WORKFLOW EXAMPLE
# ============================================
"""
WORKFLOW SETUP IN COMFYUI:

1. BASIC NODES:
   - Load Checkpoint: Select "anything-v5.safetensors"
   - Load VAE: Select "vae-ft-mse.safetensors"

2. LORA NODES:
   - Add "Load LoRA" node
   - Connect to model/clip from checkpoint
   - Select LoRA: "detail-tweaker.safetensors"
   - Strength: 0.5-0.8
   - Chain multiple LoRAs if needed

3. PROMPTS WITH EMBEDDINGS:
   Positive: "1girl, anime, masterpiece, best quality, detailed face,
             beautiful eyes, long hair, school uniform, outdoor"

   Negative: "EasyNegative, bad-hands-5, bad-artist-anime,
             lowres, bad anatomy, text, watermark, signature"

4. SAMPLER SETTINGS:
   - Steps: 25-30
   - CFG Scale: 7-9
   - Sampler: DPM++ 2M Karras or Euler a
   - Scheduler: Karras or Normal

5. REFINEMENT (Hi-Res Fix):
   - Add "Upscale Image (using Model)" node
   - Model: "4x-AnimeSharp.pth"
   - Add "KSampler" for refinement pass
   - Denoise: 0.4-0.6
   - Steps: 15-20

6. WORKFLOW FLOW:
   Checkpoint → LoRA → CLIP Encode (with embeddings) → KSampler →
   VAE Decode → Upscale → KSampler (refine) → VAE Decode → Save
"""

# ============================================
# PYTHON API WORKFLOW EXAMPLE
# ============================================
import json

def create_advanced_workflow(
    prompt="1girl, anime, masterpiece, best quality",
    negative="EasyNegative, bad-hands-5, bad-artist-anime",
    width=512,
    height=768,
    steps=28,
    cfg=8,
    seed=-1
):
    """
    Advanced workflow with LoRA, embeddings, and hi-res fix
    """
    workflow = {
        # Load Checkpoint
        "1": {
            "inputs": {"ckpt_name": "anything-v5.safetensors"},
            "class_type": "CheckpointLoaderSimple"
        },
        # Load LoRA
        "2": {
            "inputs": {
                "lora_name": "detail-tweaker.safetensors",
                "strength_model": 0.7,
                "strength_clip": 0.7,
                "model": ["1", 0],
                "clip": ["1", 1]
            },
            "class_type": "LoraLoader"
        },
        # Positive Prompt
        "3": {
            "inputs": {"text": prompt, "clip": ["2", 1]},
            "class_type": "CLIPTextEncode"
        },
        # Negative Prompt (with embeddings)
        "4": {
            "inputs": {"text": negative, "clip": ["2", 1]},
            "class_type": "CLIPTextEncode"
        },
        # Empty Latent
        "5": {
            "inputs": {
                "width": width,
                "height": height,
                "batch_size": 1
            },
            "class_type": "EmptyLatentImage"
        },
        # First Pass KSampler
        "6": {
            "inputs": {
                "seed": seed,
                "steps": steps,
                "cfg": cfg,
                "sampler_name": "dpmpp_2m_sde",
                "scheduler": "karras",
                "denoise": 1.0,
                "model": ["2", 0],
                "positive": ["3", 0],
                "negative": ["4", 0],
                "latent_image": ["5", 0]
            },
            "class_type": "KSampler"
        },
        # VAE Decode
        "7": {
            "inputs": {
                "samples": ["6", 0],
                "vae": ["1", 2]
            },
            "class_type": "VAEDecode"
        },
        # Upscale Image
        "8": {
            "inputs": {
                "upscale_model_name": "4x-AnimeSharp.pth",
                "image": ["7", 0]
            },
            "class_type": "ImageUpscaleWithModel"
        },
        # Scale Down (optional, to control final size)
        "9": {
            "inputs": {
                "upscale_method": "lanczos",
                "width": width * 2,
                "height": height * 2,
                "crop": "disabled",
                "image": ["8", 0]
            },
            "class_type": "ImageScale"
        },
        # VAE Encode for refinement
        "10": {
            "inputs": {
                "pixels": ["9", 0],
                "vae": ["1", 2]
            },
            "class_type": "VAEEncode"
        },
        # Refinement KSampler
        "11": {
            "inputs": {
                "seed": seed,
                "steps": 20,
                "cfg": cfg,
                "sampler_name": "dpmpp_2m_sde",
                "scheduler": "karras",
                "denoise": 0.5,
                "model": ["2", 0],
                "positive": ["3", 0],
                "negative": ["4", 0],
                "latent_image": ["10", 0]
            },
            "class_type": "KSampler"
        },
        # Final VAE Decode
        "12": {
            "inputs": {
                "samples": ["11", 0],
                "vae": ["1", 2]
            },
            "class_type": "VAEDecode"
        },
        # Save Image
        "13": {
            "inputs": {
                "filename_prefix": "ComfyUI_Anime",
                "images": ["12", 0]
            },
            "class_type": "SaveImage"
        }
    }
    return workflow

print("\n" + "=" * 60)
print("✅ Setup Complete!")
print("=" * 60)
print("\n📝 What's Included:")
print("  • Checkpoint Models: Anything V5, Counterfeit V3")
print("  • LoRAs: Detail Tweaker, Add Detail, Anime Lineart")
print("  • Embeddings: EasyNegative, Bad Hands 5, Bad Artist Anime")
print("  • Upscalers: 4x AnimeSharp, 4x Foolhardy Remacri")
print("\n💡 Usage Tips:")
print("  • Use embeddings in negative prompt by name")
print("  • LoRA strength: 0.5-0.8 for subtle effects")
print("  • Hi-res fix denoise: 0.4-0.6 for refinement")
print("  • Best samplers: DPM++ 2M Karras, Euler a")
print("=" * 60)

In [ ]:
# ComfyUI Manager (install other nodes easily)
!cd ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager

# ControlNet
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux

# IP-Adapter
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus

# Ultimate SD Upscale
!git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale

# Efficiency Nodes (all-in-one quality nodes)
!git clone https://github.com/jags111/efficiency-nodes-comfyui

# WAS Node Suite (utility nodes)
!git clone https://github.com/WASasquatch/was-node-suite-comfyui